In [3]:
import os
import datetime as dt
# import wget
import zipfile
from tqdm import tqdm
import pandas as pd
import xml.etree.ElementTree as ET
import glob
import numpy as np
import pprint

from requests.auth import HTTPBasicAuth,HTTPDigestAuth
import requests

In [4]:
df = pd.read_csv('../dataset/cortellis/drug_info_df.csv')

In [6]:
df.shape

(2578, 25)

In [7]:
df.head()

,drug_id,drug_name,drug_synonyms_value,drug_synonyms_type,therapy_areas,drug_primary_action_value,drug_primary_action_id,drug_secondary_action_value,drug_secondary_action_id,drug_primary_indication_value,...,drug_secondary_company_value,drug_secondary_company_id,drug_company_originator_value,drug_company_originator_id,drug_brand_names,drug_generic_names,molecular_mechanism_value,molecular_mechanism_id,cellular_mechanism_value,cellular_mechanism_id
0,100013,"asparaginase, Medac","asparaginase, Medac|Spectrila",Trade Name,Cancer,Asparaginase stimulator,517,Apoptosis stimulator|Anticancer,1589|1545,Acute lymphoblastic leukemia,...,NaN,NaN,Medac GmbH,17967,NaN,NaN,NaN,NaN,NaN,NaN
1,100057,MRx-518,"live biotherapeutic preparation (MicroRx, canc...",Research Code|Research Code,Cancer,NaN,NaN,Microbiome modulator|Immunostimulant|Anticancer,173474|393|1545,Pancreas tumor|Prostate tumor|Stage IV melanom...,...,NaN,NaN,4D pharma plc,1093689,NaN,NaN,NaN,NaN,NaN,NaN
2,100070,naporafenib,"LXH-254|pan-RAF inhibitor (oral/tablet, advanc...",Research Code|PINN,Cancer,Raf 1 protein kinase inhibitor|Raf B protein k...,4550|4548,Anticancer protein kinase inhibitor,62255,Stage IV melanoma|Metastatic colorectal cancer...,...,NaN,NaN,Novartis Pharmaceuticals Corp,24818,NaN,NaN,NaN,NaN,NaN,NaN
3,100109,"tirapazamine (hepatocellular carcinoma), Tecli...",tirapazamine|tirapazamine (hepatocellular carc...,USAN|INN,Cancer,NaN,NaN,Bioreducible cytotoxin|Anticancer,1585|1545,Neuroendocrine tumor|Hepatocellular carcinoma|...,...,NaN,NaN,Teclison Ltd,1116794,NaN,NaN,NaN,NaN,NaN,NaN
4,100129,"topotecan hydrochloride (oral, small-cell lung...",Hycamtin|topotecan hydrochloride|topotecan|top...,Trade Name|USAN|BAN|INN,Cancer,Topoisomerase I inhibitor,141,Anticancer,1545,Small-cell lung cancer,...,GlaxoSmithKline plc,28355,GlaxoSmithKline plc,28355,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
sample_drug = df['drug_id'].sample(1)

In [11]:
sample_drug.iloc[0]

120013

In [12]:
url = 'https://api.cortellis.com/api-ws/ws/rs/drugs-v2/drug/' + str(sample_drug.iloc[0])
response = requests.get(url, auth=HTTPDigestAuth('bmstrial01','A6HEW41CBIK5PFJ9'))
response.content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><drugRecordOutput id="120013"><DrugName>ICG-134</DrugName><DrugSynonyms><Name><Value>CD19 CARvac T cells</Value></Name><Name><Value>anti-CD19 CAR T-cell therapy + tumor vaccine (CARvac, B-cell lymphoma/B cell leukemia), iCell Gene Therapeutics/iCAR Bio</Value></Name><Name><Value>ICG-134</Value><Types><Type>Research Code</Type></Types></Name></DrugSynonyms><CompanyOriginator id="1124295">iCell Gene Therapeutics</CompanyOriginator><CompaniesPrimary><Company id="1195729">iCar Bio Therapeutics Ltd</Company><Company id="1124295">iCell Gene Therapeutics</Company></CompaniesPrimary><CrossReferences><SourceEntity id="1124295" type="Company"><TargetEntity id="5050988496" type="organizationId">Icell Gene Therapeutics LLC</TargetEntity></SourceEntity><SourceEntity id="1195729" type="Company"><TargetEntity id="5071483129" type="organizationId">iCAR Bio Therapeutics Ltd</TargetEntity></SourceEntity><SourceEntity id="199" type="ciIndication"><

In [13]:
root = ET.fromstring(response.content)

In [15]:
temp = root.find('DevelopmentProfile')

In [20]:
# convert xml to nested dictionary, potentially to json
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict = smart_update(aDict, dict(element.items()))
#                     aDict.update(dict(element.items()))
#                 self.update({element.tag: aDict})
                self = smart_update(self, {element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
#                 self.update({element.tag: dict(element.items())})
                self = smart_update(self, {element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
#                 self.update({element.tag: element.text})
                sel = smart_update(self, {element.tag: element.text})
#     @staticmethod
#     def smart_update(self, new_dict):
#         for key in new_dict.keys():
#             if key not in self:
#                 self[key] = new_dict[key]
#             elif isinstance(self[key], list):
#                 self[key].append(new_dict[key])
#             else:
#                 self[key] = [self[key], new_dict[key]]
def smart_update(ori_dict, new_dict):
    for key in new_dict.keys():
        if key not in ori_dict:
            ori_dict[key] = new_dict[key]
        elif isinstance(ori_dict[key], list):
            ori_dict[key].append(new_dict[key])
        else:
            ori_dict[key] = [ori_dict[key], new_dict[key]]
    return ori_dict

In [21]:
xmldict = XmlDictConfig(root)

In [23]:
xmldict['DevelopmentProfile']

{'Summary': {'displayLabel': 'Summary',
  'value': '<Summary><para><ulink linkID="1124295" linkType="Company">iCell Gene Therapeutics</ulink> and its subsidiary  <ulink linkID="1195729" linkType="Company">iCAR Bio Therapeutics</ulink>  are developing \tICG-134, a CD19 CARvac T cell therapy, comprising T cells genetically engineered  (via lentiviral transduction)  to express chimeric antigen receptors   (CAR) specific for CD19 in combination with a tumor-vaccine effect,   using its CARvac platform, for the potential treatment of  relapsed or refractory B cell malignancies including B cell lymphoma and B cell leukemia    [<ulink linkID="2216937" linkType="Reference">2216937</ulink>], [<ulink linkID="2195639" linkType="Reference">2195639</ulink>]. In February  2018, an early    phase I study was initiated   [<ulink linkID="2216937" linkType="Reference">2216937</ulink>], [<ulink linkID="2195639" linkType="Reference">2195639</ulink>]. In November 2019, data from the study were expected in 2